In [ ]:
#Question-3
import numpy as np

def compute_manipulator_jacobian(dh_parameters, joint_types=None, joint_angles=None):
    """
    Compute manipulator Jacobian, end-effector position, and velocity.

    Args:
    dh_parameters (list): List of DH parameters in the format [d, theta, a, alpha].
    joint_types (list, optional): List of joint types ('R' for revolute, 'P' for prismatic).
        Defaults to None, assuming all joints are revolute.
    joint_angles (list, optional): List of joint angles in radians. Defaults to None.

    Returns:
    (a) J: Complete manipulator Jacobian.
    (b) end_effector_pos: End-effector position as a 3x1 numpy array.
    (c) end_effector_vel: End-effector velocity as a 3x1 numpy array.
    """
    if joint_types is None:
        joint_types = ['R'] * len(dh_parameters)

    if joint_angles is None:
        joint_angles = [0.0] * len(dh_parameters)

    # Initialize transformation matrix
    T = np.eye(4)

    # Initialize Jacobian matrix
    J = np.zeros((6, len(dh_parameters)))

    for i in range(len(dh_parameters)):
        d, theta, a, alpha = dh_parameters[i]
        q = joint_angles[i]

        if joint_types[i] == 'R':
            # Revolute joint
            T_i = np.array([
                [np.cos(q), -np.sin(q) * np.cos(alpha), np.sin(q) * np.sin(alpha), a * np.cos(q)],
                [np.sin(q), np.cos(q) * np.cos(alpha), -np.cos(q) * np.sin(alpha), a * np.sin(q)],
                [0, np.sin(alpha), np.cos(alpha), d],
                [0, 0, 0, 1]
            ])
            J_i = np.array([
                [-np.sin(q), np.cos(q) * np.cos(alpha), -np.cos(q) * np.sin(alpha), 0, 0, 0],
                [np.cos(q), np.sin(q) * np.cos(alpha), -np.sin(q) * np.sin(alpha), 0, 0, 0],
                [0, -np.sin(alpha), -np.cos(alpha), 0, 0, 0],
                [0, 0, 0, -np.sin(q), np.cos(q) * np.cos(alpha), -np.cos(q) * np.sin(alpha)],
                [0, 0, 0, np.cos(q), np.sin(q) * np.cos(alpha), -np.sin(q) * np.sin(alpha)],
                [0, 0, 0, 0, -np.sin(alpha), -np.cos(alpha)]
            ])
        elif joint_types[i] == 'P':
            # Prismatic joint
            T_i = np.array([
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, d + q],
                [0, 0, 0, 1]
            ])
            J_i = np.array([
                [0, 0, 0, np.cos(alpha), 0, 0],
                [0, 0, 0, np.sin(alpha), 0, 0],
                [0, 0, 0, 0, 0, 1]
            ])
        else:
            raise ValueError("Joint type must be 'R' (revolute) or 'P' (prismatic).")

        T = np.dot(T, T_i)
        J[:, i] = np.dot(J_i, np.array([0, 0, 0, 1]))

    # Extract end-effector position
    end_effector_pos = T[:3, 3]

    # Extract end-effector velocity
    end_effector_vel = J[:3, :] @ np.array(joint_angles)

    return J, end_effector_pos, end_effector_vel

# Example usage:
if __name__ == "__main__":
    # Example DH parameters: [d, theta, a, alpha]
    dh_parameters = [
        [0, np.pi / 2, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 0, np.pi / 2]
    ]

    # Example joint types: 'R' for revolute, 'P' for prismatic
    joint_types = ['R', 'R', 'P']

    # Example joint angles in radians
    joint_angles = [0.1, 0.2, 0.3]

    J, pos, vel = compute_manipulator_jacobian(dh_parameters, joint_types, joint_angles)
    print("Manipulator Jacobian:")
    print(J)
    print("End-effector position:")
    print(pos)
    print("End-effector velocity:")
    print(vel)


In [ ]:
#Question-4
import numpy as np

# Function to manually calculate Jacobian for Stanford manipulator (RRP)
def manual_stanford_jacobian(theta1, d2, theta3):
    J = np.array([
        [-np.sin(theta1) * d2, 0, -np.cos(theta1) * np.sin(theta3)],
        [np.cos(theta1) * d2, 0, -np.sin(theta1) * np.sin(theta3)],
        [0, 1, np.cos(theta3)],
        [0, 0, 0],
        [0, 0, 0],
        [1, 0, 0]
    ])
    return J

# Function to manually calculate end-effector position for Stanford manipulator (RRP)
def manual_stanford_position(theta1, d2, theta3):
    x = np.cos(theta1) * d2 * np.cos(theta3)
    y = np.sin(theta1) * d2 * np.cos(theta3)
    z = d2 * np.sin(theta3)
    return np.array([x, y, z])

# Function to manually calculate end-effector velocity for Stanford manipulator (RRP)
def manual_stanford_velocity(theta1, d2, theta3, theta1_dot, d2_dot, theta3_dot):
    x_dot = -d2_dot * np.sin(theta1) * np.cos(theta3) + d2 * np.sin(theta1) * np.sin(theta3) * theta3_dot
    y_dot = d2_dot * np.cos(theta1) * np.cos(theta3) - d2 * np.cos(theta1) * np.sin(theta3) * theta3_dot
    z_dot = d2_dot * np.sin(theta3) + d2 * np.cos(theta3) * theta3_dot
    return np.array([x_dot, y_dot, z_dot])

# Configuration 1
theta1 = np.radians(45)
d2 = 0.5
theta3 = np.radians(30)

# Calculate using the code
J_code, pos_code, vel_code = compute_manipulator_jacobian([[0, theta1, 0, 0.1], [d2, 0, 0, 0.1], [0, theta3, 0, 0.1]], ['R', 'P', 'R'], [0, 0, 0])

# Calculate manually
J_manual = manual_stanford_jacobian(theta1, d2, theta3)
pos_manual = manual_stanford_position(theta1, d2, theta3)
vel_manual = manual_stanford_velocity(theta1, d2, theta3, 0, 0, 0)

# Compare the results
print("Stanford Manipulator (RRP) - Configuration 1:")
print("Jacobian (Code):")
print(J_code)
print("Jacobian (Manual):")
print(J_manual)
print("End-effector Position (Code):")
print(pos_code)
print("End-effector Position (Manual):")
print(pos_manual)
print("End-effector Velocity (Code):")
print(vel_code)
print("End-effector Velocity (Manual):")
print(vel_manual)

# Configuration 2
theta1 = np.radians(60)
d2 = 0.7
theta3 = np.radians(45)

# Calculate using the code
J_code, pos_code, vel_code = compute_manipulator_jacobian([[0, theta1, 0, 0.1], [d2, 0, 0, 0.1], [0, theta3, 0, 0.1]], ['R', 'P', 'R'], [0, 0, 0])

# Calculate manually
J_manual = manual_stanford_jacobian(theta1, d2, theta3)
pos_manual = manual_stanford_position(theta1, d2, theta3)
vel_manual = manual_stanford_velocity(theta1, d2, theta3, 0, 0, 0)

# Compare the results
print("\nStanford Manipulator (RRP) - Configuration 2:")
print("Jacobian (Code):")
print(J_code)
print("Jacobian (Manual):")
print(J_manual)
print("End-effector Position (Code):")
print(pos_code)
print("End-effector Position (Manual):")
print(pos_manual)
print("End-effector Velocity (Code):")
print(vel_code)
print("End-effector Velocity (Manual):")
print(vel_manual)


# For SCARA Manipulator
# Function to manually calculate Jacobian for SCARA manipulator
def manual_scara_jacobian(theta1, d2, theta3):
    J = np.array([
        [-d2 * np.sin(theta1), 0, 0],
        [d2 * np.cos(theta1), 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [1, 0, 0]
    ])
    return J

# Function to manually calculate end-effector position for SCARA manipulator
def manual_scara_position(theta1, d2, theta3):
    x = d2 * np.cos(theta1)
    y = d2 * np.sin(theta1)
    z = 0
    return np.array([x, y, z])

# Function to manually calculate end-effector velocity for SCARA manipulator
def manual_scara_velocity(theta1, d2, theta3, theta1_dot, d2_dot, theta3_dot):
    x_dot = -d2 * np.sin(theta1) * theta1_dot
    y_dot = d2 * np.cos(theta1) * theta1_dot
    z_dot = 0
    return np.array([x_dot, y_dot, z_dot])

# Configuration 1
theta1 = np.radians(30)
d2 = 0.4
theta3 = np.radians(45)

# Calculate using the code
J_code, pos_code, vel_code = compute_manipulator_jacobian([[0, theta1, 0, 0.1], [d2, 0, 0, 0.1], [0, theta3, 0, 0.1]], ['R', 'P', 'R'], [0, 0, 0])

# Calculate manually
J_manual = manual_scara_jacobian(theta1, d2, theta3)
pos_manual = manual_scara_position(theta1, d2, theta3)
vel_manual = manual_scara_velocity(theta1, d2, theta3, 0, 0, 0)

# Compare the results
print("\nSCARA Manipulator - Configuration 1:")
print("Jacobian (Code):")
print(J_code)
print("Jacobian (Manual):")
print(J_manual)
print("End-effector Position (Code):")
print(pos_code)
print("End-effector Position (Manual):")
print(pos_manual)
print("End-effector Velocity (Code):")
print(vel_code)
print("End-effector Velocity (Manual):")
print(vel_manual)

# Configuration 2
theta1 = np.radians(60)
d2 = 0.6
theta3 = np.radians(30)

# Calculate using the code
J_code, pos_code, vel_code = compute_manipulator_jacobian([[0, theta1, 0, 0.1], [d2, 0, 0, 0.1], [0, theta3, 0, 0.1]], ['R', 'P', 'R'], [0, 0, 0])

# Calculate manually
J_manual = manual_scara_jacobian(theta1, d2, theta3)
pos_manual = manual_scara_position(theta1, d2, theta3)
vel_manual = manual_scara_velocity(theta1, d2, theta3, 0, 0, 0)

# Compare the results
print("\nSCARA Manipulator - Configuration 2:")
print("Jacobian (Code):")
print(J_code)
print("Jacobian (Manual):")
print(J_manual)
print("End-effector Position (Code):")
print(pos_code)
print("End-effector Position (Manual):")
print(pos_manual)
print("End-effector Velocity (Code):")
print(vel_code)
print("End-effector Velocity (Manual):")
print(vel_manual)



In [ ]:
#Question-11
import sympy as sp

# Define symbolic variables
q1, q2, q3 = sp.symbols('q1 q2 q3')
q1_dot, q2_dot, q3_dot = sp.symbols('q1_dot q2_dot q3_dot')
q1_ddot, q2_ddot, q3_ddot = sp.symbols('q1_ddot q2_ddot q3_ddot')
tau1, tau2, tau3 = sp.symbols('tau1 tau2 tau3')

# Define D(q) and V(q) functions (replace with your robot's functions)
# For this example, we assume a simple diagonal inertia matrix D(q) and
# velocity-dependent forces V(q).
def D(q):
    return sp.diag(q1**2, q2**2, q3**2)

def V(q, q_dot):
    return sp.Matrix([q1_dot**2, q2_dot**2, q3_dot**2])

# Define gravity vector (replace with your robot's gravity model)
# For this example, we assume a 3D vector [0, 0, -g].
g = sp.symbols('g')
gravity_vector = sp.Matrix([0, 0, -g])

# Define symbolic joint velocity vector q_dot
q_dot = sp.Matrix([q1_dot, q2_dot, q3_dot])

# Define the Lagrangian L(q, q_dot)
L = sp.Rational(1, 2) * q_dot.dot(D([q1, q2, q3])).dot(q_dot) - V([q1, q2, q3], q_dot).dot(q_dot) + gravity_vector.dot(q_dot)

# Compute the equations of motion using the Euler-Lagrange equation
# E = d/dt (∂L/∂q_dot) - ∂L/∂q - generalized forces (tau)
E = sp.Eq(sp.diff(L, q1_dot).diff(q1) + sp.diff(L, q1_dot).diff(q2) + sp.diff(L, q1_dot).diff(q3) - sp.diff(L, q1) - tau1, 0)
E += sp.Eq(sp.diff(L, q2_dot).diff(q1) + sp.diff(L, q2_dot).diff(q2) + sp.diff(L, q2_dot).diff(q3) - sp.diff(L, q2) - tau2, 0)
E += sp.Eq(sp.diff(L, q3_dot).diff(q1) + sp.diff(L, q3_dot).diff(q2) + sp.diff(L, q3_dot).diff(q3) - sp.diff(L, q3) - tau3, 0)

# Simplify the equations
simplified_eqs = sp.simplify(E)

# Display the simplified equations of motion
print("Equations of Motion:")
for eq in simplified_eqs:
    print(eq)


In [ ]:
#Question-12
import numpy as np

def inverse_position_kinematics(x, y, z, l1, l2):
    # Calculate the prismatic joint extension d2 using the end-effector position
    d2 = np.sqrt(x**2 + y**2 + z**2) - l1

    # Calculate the joint angles θ1 and θ3 using trigonometry
    theta1 = np.arctan2(y, x)
    theta3 = np.arctan2(z, np.sqrt(x**2 + y**2))

    # Convert angles to degrees if needed
    theta1_deg = np.degrees(theta1)
    theta3_deg = np.degrees(theta3)

    return theta1_deg, d2, theta3_deg

# Define the desired end-effector position (x, y, z) and link lengths (l1, l2)
x_desired = 0.5  # Replace with your desired x-coordinate
y_desired = 0.3  # Replace with your desired y-coordinate
z_desired = 0.2  # Replace with your desired z-coordinate
l1_length = 0.4   # Length of link 1
l2_length = 0.3   # Length of link 2

# Call the inverse position kinematics subroutine
theta1_deg, d2_value, theta3_deg = inverse_position_kinematics(x_desired, y_desired, z_desired, l1_length, l2_length)

# Print the results
print("Joint Angles (θ1 and θ3 in degrees):")
print("θ1 =", theta1_deg, "degrees")
print("θ3 =", theta3_deg, "degrees")
print("Prismatic Joint Extension (d2):")
print("d2 =", d2_value)


In [ ]:
#Question-13
import numpy as np

def inverse_position_kinematics_scara(x, y, l1, l2):
    # Calculate the distance from the origin to the end-effector position
    r = np.sqrt(x**2 + y**2)

    # Calculate the angle θ2 using the law of cosines
    cos_theta2 = (r**2 - l1**2 - l2**2) / (2 * l1 * l2)
    sin_theta2 = np.sqrt(1 - cos_theta2**2)
    theta2 = np.arctan2(sin_theta2, cos_theta2)

    # Calculate the angle θ1 using trigonometry
    theta1 = np.arctan2(y, x) - np.arctan2(l2 * sin_theta2, l1 + l2 * cos_theta2)

    # Convert angles to degrees if needed
    theta1_deg = np.degrees(theta1)
    theta2_deg = np.degrees(theta2)

    return theta1_deg, theta2_deg

# Define the desired end-effector position (x, y) and link lengths (l1, l2)
x_desired = 0.4  # Replace with your desired x-coordinate
y_desired = 0.3  # Replace with your desired y-coordinate
l1_length = 0.2  # Length of link 1
l2_length = 0.2  # Length of link 2

# Call the inverse position kinematics subroutine
theta1_deg, theta2_deg = inverse_position_kinematics_scara(x_desired, y_desired, l1_length, l2_length)

# Print the results
print("Joint Angles (θ1 and θ2 in degrees):")
print("θ1 =", theta1_deg, "degrees")
print("θ2 =", theta2_deg, "degrees")


In [ ]:
#Question-14
import numpy as np

def calculate_joint_velocities(Jacobian, end_effector_velocities):
    try:
        # Check if the Jacobian matrix is invertible
        if np.linalg.matrix_rank(Jacobian) < min(Jacobian.shape):
            raise ValueError("Jacobian matrix is not invertible.")

        # Calculate joint velocities using the Jacobian
        joint_velocities = np.linalg.inv(Jacobian).dot(end_effector_velocities)

        return joint_velocities

    except Exception as e:
        return None, str(e)

# Define the Jacobian matrix and end-effector velocities (dx, dy, dz)
# Replace with your specific Jacobian and velocities
Jacobian = np.array([[0.1, 0.2, 0.3],
                     [0.4, 0.5, 0.6],
                     [0.7, 0.8, 0.9]])

end_effector_velocities = np.array([0.1, 0.2, 0.3])

# Call the subroutine to calculate joint velocities
joint_velocities = calculate_joint_velocities(Jacobian, end_effector_velocities)

if joint_velocities is not None:
    print("Joint Velocities:")
    print(joint_velocities)
else:
    print("Error:", joint_velocities)


In [ ]:
#Question-17
import numpy as np
from scipy.spatial.transform import Rotation as R

def spherical_wrist_inverse_kinematics(R):
    # Extract the rotation matrix components
    r11, r12, r13 = R[0, 0], R[0, 1], R[0, 2]
    r21, r22, r23 = R[1, 0], R[1, 1], R[1, 2]
    r31, r32, r33 = R[2, 0], R[2, 1], R[2, 2]

    # Calculate joint angles θ1, θ2, θ3
    theta1 = np.arctan2(r21, r11)
    theta2 = np.arctan2(-r31, np.sqrt(r11**2 + r21**2))
    theta3 = np.arctan2(r32, r33)

    # Convert angles to degrees if needed
    theta1_deg = np.degrees(theta1)
    theta2_deg = np.degrees(theta2)
    theta3_deg = np.degrees(theta3)

    return theta1_deg, theta2_deg, theta3_deg

# Define the desired wrist orientation as a rotation matrix
# Replace with your desired rotation matrix
desired_rotation_matrix = np.array([[0.866, -0.5, 0],
                                    [0.5, 0.866, 0],
                                    [0, 0, 1]])

# Call the subroutine to calculate joint angles
theta1_deg, theta2_deg, theta3_deg = spherical_wrist_inverse_kinematics(desired_rotation_matrix)

# Print the results
print("Joint Angles (θ1, θ2, θ3 in degrees):")
print("θ1 =", theta1_deg, "degrees")
print("θ2 =", theta2_deg, "degrees")
print("θ3 =", theta3_deg, "degrees")


In [ ]:
#Question-18
import numpy as np

def forward_kinematics_PPP(d1, d2, d3):
    # Define DH parameters for the PPP configuration
    a1, α1, d1_offset, θ1 = 0, 0, 0, 0
    a2, α2, d2_offset, θ2 = 0, 0, 0, 0
    a3, α3, d3_offset, θ3 = 0, 0, 0, 0

    # Transformation matrices for the three prismatic joints
    T1 = np.array([[np.cos(θ1), -np.sin(θ1), 0, 0],
                   [np.sin(θ1), np.cos(θ1), 0, 0],
                   [0, 0, 1, d1 + d1_offset],
                   [0, 0, 0, 1]])

    T2 = np.array([[np.cos(θ2), -np.sin(θ2), 0, 0],
                   [np.sin(θ2), np.cos(θ2), 0, 0],
                   [0, 0, 1, d2 + d2_offset],
                   [0, 0, 0, 1]])

    T3 = np.array([[np.cos(θ3), -np.sin(θ3), 0, 0],
                   [np.sin(θ3), np.cos(θ3), 0, 0],
                   [0, 0, 1, d3 + d3_offset],
                   [0, 0, 0, 1]])

    # Compute the end-effector transformation matrix
    T_end_effector = np.dot(np.dot(T1, T2), T3)

    # Extract end-effector position and orientation
    end_effector_position = T_end_effector[:3, 3]
    end_effector_orientation = T_end_effector[:3, :3]

    return T_end_effector, end_effector_position, end_effector_orientation

def jacobian_PPP(d1, d2, d3):
    # Define symbolic variables for joint extensions
    d1_symbolic = d1
    d2_symbolic = d2
    d3_symbolic = d3

    # Calculate the Jacobian matrix using symbolic variables
    J = np.array([[0, 0, 0],
                  [0, 0, 0],
                  [1, 1, 1]])

    # Replace symbolic variables with numerical values
    J = J.subs([(d1_symbolic, d1), (d2_symbolic, d2), (d3_symbolic, d3)])

    return J

# Representative numerical values for joint extensions (d1, d2, d3)
d1_value = 0.1  # Replace with your desired value
d2_value = 0.2  # Replace with your desired value
d3_value = 0.3  # Replace with your desired value

# Calculate forward kinematics
T_end_effector, end_effector_position, end_effector_orientation = forward_kinematics_PPP(d1_value, d2_value, d3_value)

# Calculate the Jacobian matrix
Jacobian = jacobian_PPP(d1_value, d2_value, d3_value)

# Print results
print("Forward Kinematics:")
print("End-Effector Position:")
print(end_effector_position)
print("End-Effector Orientation (Rotation Matrix):")
print(end_effector_orientation)
print("\nJacobian Matrix:")
print(Jacobian)
